In [239]:
# Biblioteki wejściowe
import pandas as pd
import numpy as np
import numpy_financial as npf
import posixpath as os
import pathlib
from pathlib import Path
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"
from plotly.subplots import make_subplots
from pandas.tseries.offsets import MonthEnd

pd.options.display.max_rows=50000
pd.options.display.max_colwidth=100
pd.options.display.precision=2
pd.options.display.max_columns=500
pd.options.display.float_format='{0:,.2f}'.format

ASSETS = Path(r'C:\Users\DW\Box\KOPALNIA\FINANSE\FS_LIBERTAS\ASSETS')
IMAGES = Path(r'C:\Users\DW\Box\KOPALNIA\FINANSE\FS_LIBERTAS\IMAGES')

In [240]:
#------------------------------------------------------PARAMETRY MODELU-------------------------------------------------------------

#------------------Stopy-----FX-----------------------------------------------------------------------------------------------------

FX_EUR=4.72
WIBOR_1M=7.06
DISCOUNT_RATE=9.0


# Zakres czasowy -------------------------------------------------------------------------------------------------------------------

DATE_START=pd.to_datetime('2023-01-01')
DATE_TENOR=144
LEASING_TENOR=60

# Sezonowość -----------------------------------------------------------------------------------------------------------------------

t_sezon=pd.DataFrame([
    [1,4],
    [2,4],
    [3,5],
    [4,10],
    [5,10],
    [6,13],
    [7,13],
    [8,13],
    [9,10],
    [10,10],
    [11,4],
    [12,4],
],columns=['M','PROD_PROC'])

# Produkcja -----------------------------------------------------------------------------------------------------------------------

PROD_START=900000
PROD_CAGR=0.02

# Wydajność koparki ----------------------------------------------------------------------------------------------------------------

DIGGER_BUCKET=4.14 # pojemność łyżki w tonach
DIGGER_BUCKET_TIME=0.42 # czas załadunku łyżki w minutach na cykl
HAULER_LOAD=23 # załadunek wozidła w tonach
HAULER_QUE_TIME=1 # manewrowanie i oczekiwanie na załadunek w minutach

# Wydajność wozidła ----------------------------------------------------------------------------------------------------------------

HAULER_LOAD_TIME=(HAULER_LOAD/DIGGER_BUCKET)*DIGGER_BUCKET_TIME
HAULER_TOBUFOR_TIME=3
HAULER_FROMBUFOR_TIME=2.5
HAULER_DUMP_TIME=0.2 # wyładunek
HAULER_AMOUNT=3
DIGGER_AMOUNT=1
LOADER_AMOUNT=1

# Efektywność ----------------------------------------------------------------------------------------------------------------------

ALLIN_EFFICIENCY=0.9
HAULER_ADD_TIME=250 # w godzinach na miesiąc średnie zapotrzebowanie
DIGGER_ADD_TIME=100 # w godzinach na miesiąc średnie zapotrzebowanie

# Ceny jednostkowe -----------------------------------------------------------------------------------------------------------------

DIGGER_AND_HAULER_PRICE=3.99 #Stawka Waśpol 3.99
HAULER_TIME_PRICE=240
DIGGER_TIME_PRICE=340
LOADER_INCOME_MARGIN=0.09

# Koszt TNMR ----------------------------------------------------------------------------------------------------------------------

HAULER_TNMR_UNIT_COST=38
DIGGER_TNMR_UNIT_COST=36
LOADER_TNMR_UNIT_COST=37

# Koszt Paliwa --------------------------------------------------------------------------------------------------------------------

HAULER_FUEL_CONSUMPTION=14.48 #L/H
DIGGER_FUEL_CONSUMPTION=45.37 #L/H
LOADER_FUEL_CONSUMPTION=45.37 #L/H
FUEL_PRICE=6.67

# Koszty pozostałe ----------------------------------------------------------------------------------------------------------------

HAULER_TYRES_UNIT_COST=63200
LOADER_TYRES_UNIT_COST=125600
DIGGER_BUCKET_UNIT_COST=23200 # co roku
LOADER_BUCKET_UNIT_COST=50000 # co roku

HAULER_SOUNDRY_COSTS=1650
DIGGER_SOUNDRY_COSTS=1250
LOADER_SOUNDRY_COSTS=650

# Koszty pracowników ---------------------------------------------------------------------------------------------------------------
# Układ czterobrygadowy 4 x (3+1) = 16 osób

PRAC_HC=16
PRAC_SALARY=5845
PRAC_ZUS_PRC=0.2293




In [241]:
# Definicja okresu analizy 60 miesięcy

okres=pd.date_range(start=DATE_START,freq='M',periods=DATE_TENOR)
t_model=pd.DataFrame({'DATE':okres})
t_model['R']=t_model['DATE'].dt.year
t_model['M']=t_model['DATE'].dt.month
t_model['I']=t_model.index

In [242]:
# Plan produkcji

t_model['PROD_YEAR']=t_model.apply(lambda x:PROD_START if x['R']==DATE_START.year else PROD_START*(1+PROD_CAGR)**(x['R']-DATE_START.year),axis=1)
t_model=pd.merge(t_model,t_sezon,how='left',on=['M'])
t_model['PROD_MONTH']=t_model['PROD_YEAR']*t_model['PROD_PROC']/100
t_model['PROD_SHIFT']=t_model['PROD_MONTH']/(4*5*3) # produkcja na zmianę bez sobót i nadgodzin

In [243]:
# Wymagany czas pracy koparki i wozidła na zmianę

t_model['DIGGER_SHIFT_TIME']=(((t_model['PROD_SHIFT']/HAULER_LOAD)*HAULER_LOAD_TIME)/ALLIN_EFFICIENCY/60)

t_model['HAULER_SHIFT_TIME']=t_model.apply(
    lambda x:(x['PROD_SHIFT']/HAULER_LOAD/HAULER_AMOUNT)*(HAULER_QUE_TIME+HAULER_LOAD_TIME+HAULER_TOBUFOR_TIME+HAULER_FROMBUFOR_TIME+HAULER_DUMP_TIME)/ALLIN_EFFICIENCY/60 
    if (x['PROD_SHIFT']/HAULER_LOAD/HAULER_AMOUNT)*(HAULER_QUE_TIME+HAULER_LOAD_TIME+HAULER_TOBUFOR_TIME+HAULER_FROMBUFOR_TIME+HAULER_DUMP_TIME)/ALLIN_EFFICIENCY/60 > ((x['PROD_SHIFT']/HAULER_LOAD)*HAULER_LOAD_TIME)/ALLIN_EFFICIENCY/60 
    else ((x['PROD_SHIFT']/HAULER_LOAD)*HAULER_LOAD_TIME)/ALLIN_EFFICIENCY/60,axis=1)

t_model['HAULER_ADD_TIME']=t_model.apply(lambda x:HAULER_ADD_TIME if (8-x['HAULER_SHIFT_TIME'])*HAULER_AMOUNT*(3*5*4)>HAULER_ADD_TIME else (8-x['HAULER_SHIFT_TIME'])*HAULER_AMOUNT*(3*5*4), axis=1)
t_model['DIGGER_ADD_TIME']=t_model.apply(lambda x:DIGGER_ADD_TIME if (8-x['HAULER_SHIFT_TIME'])*HAULER_AMOUNT*(3*5*4)>DIGGER_ADD_TIME else (8-x['HAULER_SHIFT_TIME'])*HAULER_AMOUNT*(3*5*4), axis=1)

t_model['DIGGER_MTG']=t_model['DIGGER_SHIFT_TIME']*3*5*4+t_model['DIGGER_ADD_TIME']
t_model['HAULER_MTG']=t_model['HAULER_SHIFT_TIME']*3*5*4*HAULER_AMOUNT+t_model['HAULER_ADD_TIME']

In [244]:
# Koszt zakupu Wozidła

HAULER_CAPEX_EUR=325000
# Uwaga VFS chce zarobić na FX poprzez rozliczenie po kursie sprzedaży !!!
HAULER_DP=0.3
HAULER_RV=0.22
HAULER_MG=2.2
HAULER_INS=4.942 # proc składka za 5 lat, do rozłożenia w czasie

HAULER_CAPEX_PLN=HAULER_CAPEX_EUR*FX_EUR

HAULER_RV_PLN=HAULER_CAPEX_PLN*HAULER_RV

t_model['HAULER_CAPEX']=t_model.apply(lambda x:HAULER_CAPEX_PLN if x['I']==0 else 0,axis=1)*HAULER_AMOUNT

t_model['HAULER_DEPREC']=-t_model.apply(lambda x:0 if x['I']==0 else HAULER_CAPEX_PLN*(1)/(DATE_TENOR-1),axis=1)*HAULER_AMOUNT

t_model['HAULER_DP']=-t_model.apply(lambda x:HAULER_CAPEX_PLN*HAULER_DP if x['I']==0 else 0,axis=1)*HAULER_AMOUNT

t_model['HAULER_PPMT']=t_model.apply(lambda x:0 if (x['I']==0 or x['I']>LEASING_TENOR) else npf.ppmt((WIBOR_1M+HAULER_MG)/1200,x['I'],LEASING_TENOR,HAULER_CAPEX_PLN*(1-HAULER_DP)*HAULER_AMOUNT,-HAULER_RV_PLN*HAULER_AMOUNT),axis=1)

t_model['HAULER_IPMT']=t_model.apply(lambda x:0 if (x['I']==0 or x['I']>LEASING_TENOR) else npf.ipmt((WIBOR_1M+HAULER_MG)/1200,x['I'],LEASING_TENOR,HAULER_CAPEX_PLN*(1-HAULER_DP)*HAULER_AMOUNT,-HAULER_RV_PLN*HAULER_AMOUNT),axis=1).astype(float)


t_model['HAULER_PMT']=t_model['HAULER_IPMT']+t_model['HAULER_PPMT']

t_model['HAULER_RV']=-t_model.apply(lambda x:0 if x['I']!=LEASING_TENOR else HAULER_RV_PLN,axis=1)*HAULER_AMOUNT

t_model['HAULER_OUT']=HAULER_CAPEX_PLN*(1-HAULER_DP)*HAULER_AMOUNT+t_model['HAULER_PPMT'].cumsum()+t_model['HAULER_RV'].cumsum()

t_model['HAULER_INS']=t_model.apply(lambda x:0 if x['I']==0 else npf.pmt((WIBOR_1M+HAULER_MG)/1200,LEASING_TENOR-1,HAULER_CAPEX_PLN*HAULER_INS/100,0),axis=1)*HAULER_AMOUNT
t_model['HAULER_TNMR_COST']=-t_model['HAULER_MTG']*HAULER_TNMR_UNIT_COST
t_model['HAULER_FUEL_COST']=-t_model['HAULER_MTG']*HAULER_FUEL_CONSUMPTION*FUEL_PRICE
t_model['HAULER_TYRES_COST']=-t_model.apply(lambda x:HAULER_TYRES_UNIT_COST if x['I']%30==0 else 0, axis=1)
t_model['HAULER_SOUNDRY_COST']=-t_model.apply(lambda x:0 if x['I']==0 else HAULER_SOUNDRY_COSTS, axis=1)

In [245]:
# Koszt zakupu Koparki

DIGGER_CAPEX_EUR=377900
# Uwaga VFS chce zarobić na FX poprzez rozliczenie po kursie sprzedaży !!!
DIGGER_DP=0.3
DIGGER_RV=0.17
DIGGER_MG=2.2
DIGGER_INS=4.881 # proc składka za 5 lat, do rozłożenia w czasie

DIGGER_CAPEX_PLN=DIGGER_CAPEX_EUR*FX_EUR
DIGGER_RV_PLN=DIGGER_CAPEX_PLN*DIGGER_RV

t_model['DIGGER_CAPEX']=t_model.apply(lambda x:DIGGER_CAPEX_PLN if x['I']==0 else 0,axis=1)*DIGGER_AMOUNT
t_model['DIGGER_DEPREC']=-t_model.apply(lambda x:0 if x['I']==0 else DIGGER_CAPEX_PLN*(1)/(DATE_TENOR-1),axis=1)*DIGGER_AMOUNT
t_model['DIGGER_DP']=-t_model.apply(lambda x:DIGGER_CAPEX_PLN*DIGGER_DP if x['I']==0 else 0,axis=1)
t_model['DIGGER_PPMT']=t_model.apply(lambda x:0 if (x['I']==0 or x['I']>LEASING_TENOR-1) else npf.ppmt((WIBOR_1M+DIGGER_MG)/1200,x['I'],LEASING_TENOR-1,DIGGER_CAPEX_PLN*(1-DIGGER_DP),-DIGGER_RV_PLN),axis=1)*DIGGER_AMOUNT
t_model['DIGGER_IPMT']=t_model.apply(lambda x:0 if (x['I']==0 or x['I']>LEASING_TENOR-1) else npf.ipmt((WIBOR_1M+DIGGER_MG)/1200,x['I'],LEASING_TENOR-1,DIGGER_CAPEX_PLN*(1-DIGGER_DP),-DIGGER_RV_PLN),axis=1).astype(float)*DIGGER_AMOUNT
t_model['DIGGER_PMT']=t_model['DIGGER_IPMT']+t_model['DIGGER_PPMT']
t_model['DIGGER_RV']=-t_model.apply(lambda x:0 if x['I']!=LEASING_TENOR-1 else DIGGER_RV_PLN,axis=1)*DIGGER_AMOUNT
t_model['DIGGER_OUT']=DIGGER_CAPEX_PLN*(1-DIGGER_DP)*DIGGER_AMOUNT+t_model['DIGGER_PPMT'].cumsum()+t_model['DIGGER_RV'].cumsum()
t_model['DIGGER_INS']=t_model.apply(lambda x:0 if x['I']==0 else npf.pmt((WIBOR_1M+DIGGER_MG)/1200,LEASING_TENOR-1,DIGGER_CAPEX_PLN*DIGGER_INS/100,0),axis=1)*DIGGER_AMOUNT
t_model['DIGGER_TNMR_COST']=-t_model['DIGGER_MTG']*DIGGER_TNMR_UNIT_COST
t_model['DIGGER_FUEL_COST']=-t_model['DIGGER_MTG']*DIGGER_FUEL_CONSUMPTION*FUEL_PRICE
t_model['DIGGER_BUCKET_COST']=-t_model.apply(lambda x:DIGGER_BUCKET_UNIT_COST if x['M']==12 else 0, axis=1)
t_model['DIGGER_SOUNDRY_COST']=-t_model.apply(lambda x:0 if x['I']==0 else DIGGER_SOUNDRY_COSTS, axis=1)

In [246]:
# Koszt zakupu Ładowarki

LOADER_CAPEX_EUR=384500
# Uwaga VFS chce zarobić na FX poprzez rozliczenie po kursie sprzedaży !!!
LOADER_DP=0.3
LOADER_RV=0.19
LOADER_MG=2.2
LOADER_INS=4.87 # proc składka za 5 lat, do rozłożenia w czasie

LOADER_CAPEX_PLN=LOADER_CAPEX_EUR*FX_EUR
LOADER_RV_PLN=LOADER_CAPEX_PLN*LOADER_RV

t_model['LOADER_CAPEX']=t_model.apply(lambda x:LOADER_CAPEX_PLN if x['I']==0 else 0,axis=1)*LOADER_AMOUNT
t_model['LOADER_DEPREC']=-t_model.apply(lambda x:0 if x['I']==0 else LOADER_CAPEX_PLN*(1)/(DATE_TENOR-1),axis=1)*LOADER_AMOUNT
t_model['LOADER_DP']=-t_model.apply(lambda x:LOADER_CAPEX_PLN*LOADER_DP if x['I']==0 else 0,axis=1)
t_model['LOADER_PPMT']=t_model.apply(lambda x:0 if (x['I']==0 or x['I']>LEASING_TENOR-1) else npf.ppmt((WIBOR_1M+LOADER_MG)/1200,x['I'],LEASING_TENOR-1,LOADER_CAPEX_PLN*(1-LOADER_DP),-LOADER_RV_PLN),axis=1)*LOADER_AMOUNT
t_model['LOADER_IPMT']=t_model.apply(lambda x:0 if (x['I']==0 or x['I']>LEASING_TENOR-1) else npf.ipmt((WIBOR_1M+LOADER_MG)/1200,x['I'],LEASING_TENOR-1,LOADER_CAPEX_PLN*(1-LOADER_DP),-LOADER_RV_PLN),axis=1).astype(float)*LOADER_AMOUNT
t_model['LOADER_PMT']=t_model['LOADER_IPMT']+t_model['LOADER_PPMT']
t_model['LOADER_RV']=-t_model.apply(lambda x:0 if x['I']!=LEASING_TENOR-1 else LOADER_RV_PLN,axis=1)*LOADER_AMOUNT
t_model['LOADER_OUT']=LOADER_CAPEX_PLN*(1-LOADER_DP)*LOADER_AMOUNT+t_model['LOADER_PPMT'].cumsum()+t_model['LOADER_RV'].cumsum()
t_model['LOADER_INS']=t_model.apply(lambda x:0 if x['I']==0 else npf.pmt((WIBOR_1M+LOADER_MG)/1200,LEASING_TENOR-1,LOADER_CAPEX_PLN*LOADER_INS/100,0),axis=1)*LOADER_AMOUNT
t_model['LOADER_TNMR_COST']=-18000/60*LOADER_TNMR_UNIT_COST
t_model['LOADER_FUEL_COST']=-18000/60*LOADER_FUEL_CONSUMPTION*FUEL_PRICE
t_model['LOADER_TYRES_COST']=-t_model.apply(lambda x:125600 if x['I']%30==0 else 0, axis=1)
t_model['LOADER_BUCKET_COST']=-t_model.apply(lambda x:LOADER_BUCKET_UNIT_COST if x['M']==12 else 0, axis=1)
t_model['LOADER_SOUNDRY_COST']=-t_model.apply(lambda x:0 if x['I']==0 else LOADER_SOUNDRY_COSTS, axis=1)

In [247]:
# Kalkulacja przychodów

t_model['INCOME_SHIFT']=t_model['PROD_MONTH']*DIGGER_AND_HAULER_PRICE
t_model['INCOME_ADD_TIME']=t_model['HAULER_ADD_TIME']*HAULER_TIME_PRICE+t_model['DIGGER_ADD_TIME']*DIGGER_TIME_PRICE
t_model['INCOME_LOADER']=-(t_model['LOADER_PMT']+t_model['LOADER_INS']+t_model['LOADER_TNMR_COST']+t_model['LOADER_FUEL_COST']+t_model['LOADER_TYRES_COST']+t_model['LOADER_BUCKET_COST']+t_model['LOADER_SOUNDRY_COST'])*(1+LOADER_INCOME_MARGIN)

In [248]:
# Rachunek Wyników

# Przychody
t_model['A. PRZYCHODY NETTO']=t_model['INCOME_SHIFT']+t_model['INCOME_ADD_TIME']+t_model['INCOME_LOADER']
t_model[' 1. Przychody ze sprzedaży usług']=t_model['INCOME_SHIFT']
t_model[' 2. Przychody dodatkowe ze sprzedaży usług']=t_model['INCOME_ADD_TIME']
t_model[' 3. Przychody z wynajmu']=t_model['INCOME_LOADER']

# Koszty operacyjne
t_model['B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ']=t_model['HAULER_DEPREC']+t_model['DIGGER_DEPREC']+t_model['LOADER_DEPREC']+t_model['HAULER_FUEL_COST']+t_model['DIGGER_FUEL_COST']+t_model['LOADER_FUEL_COST']+t_model['HAULER_TYRES_COST']+t_model['LOADER_TYRES_COST']+t_model['DIGGER_BUCKET_COST']+t_model['LOADER_BUCKET_COST']+t_model['HAULER_TNMR_COST']+t_model['DIGGER_TNMR_COST']+t_model['LOADER_TNMR_COST']+t_model['HAULER_INS']+t_model['DIGGER_INS']+t_model['LOADER_INS']-t_model.apply(lambda x:0 if x['I']==0 else PRAC_HC*PRAC_SALARY, axis=1)-t_model.apply(lambda x:0 if x['I']==0 else PRAC_HC*PRAC_SALARY*PRAC_ZUS_PRC, axis=1)+t_model['HAULER_SOUNDRY_COST']+t_model['DIGGER_SOUNDRY_COST']+t_model['LOADER_SOUNDRY_COST']

t_model[' 1. Amortyzacja']=t_model['HAULER_DEPREC']+t_model['DIGGER_DEPREC']+t_model['LOADER_DEPREC']
t_model[' 2. Zużycie materiałów i energii']=t_model['HAULER_FUEL_COST']+t_model['DIGGER_FUEL_COST']+t_model['LOADER_FUEL_COST']+t_model['HAULER_TYRES_COST']+t_model['LOADER_TYRES_COST']+t_model['DIGGER_BUCKET_COST']+t_model['LOADER_BUCKET_COST']
t_model[' 3. Usługi obce']=t_model['HAULER_TNMR_COST']+t_model['DIGGER_TNMR_COST']+t_model['LOADER_TNMR_COST']+t_model['HAULER_INS']+t_model['DIGGER_INS']+t_model['LOADER_INS']
t_model[' 4. Podatki i opłaty']=0
t_model[' 5. Wynagrodzenia']=-t_model.apply(lambda x:0 if x['I']==0 else PRAC_HC*PRAC_SALARY, axis=1)
t_model[' 6. Ubezpieczenia społeczne']=-t_model.apply(lambda x:0 if x['I']==0 else PRAC_HC*PRAC_SALARY*PRAC_ZUS_PRC, axis=1)
t_model[' 7. Pozostałe koszty rodzajowe']=t_model['HAULER_SOUNDRY_COST']+t_model['DIGGER_SOUNDRY_COST']+t_model['LOADER_SOUNDRY_COST']
t_model[' 8. Wartość sprzedanych towarów i materiałów']=0

# Pozostałe przychody i koszty operacyjne
t_model['C. POZOSTAŁE PRZYCHODY OPERACYJNE']=0
t_model['D. POZOSTAŁE KOSZTY OPERACYJNE']=0

# Przychody i koszty finansowe
t_model['E. PRZYCHODY FINANSOWE']=0
t_model['F. KOSZTY FINANSOWE']=t_model['HAULER_IPMT']+t_model['DIGGER_IPMT']+t_model['LOADER_IPMT']

# Wynik Brutto

t_model['G. WYNIK BRUTTO']=t_model['A. PRZYCHODY NETTO']+t_model['B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ']+t_model['C. POZOSTAŁE PRZYCHODY OPERACYJNE']+t_model['D. POZOSTAŁE KOSZTY OPERACYJNE']+t_model['E. PRZYCHODY FINANSOWE']+t_model['F. KOSZTY FINANSOWE']

# Podatek dochodowy

#t_model['H. PODATEK DOCHODOWY']=t_model.apply(lambda x:-x['G. WYNIK BRUTTO']*0.19 if x['G. WYNIK BRUTTO']>0 else 0, axis=1)
t_model['H. PODATEK DOCHODOWY']=0
# Zysk netto

t_model['I. WYNIK NETTO']=t_model['G. WYNIK BRUTTO']+t_model['H. PODATEK DOCHODOWY']



In [249]:
# Zapotrzebowanie na kapitał obrotowy

ROTACJA_NALEŻNOŚCI=14
ROTACJA_ZAPASÓW=0
ROTACJA_ZOBOWIĄZAŃ=30

t_model['A. AKTYWA BIEŻĄCE']=(t_model['A. PRZYCHODY NETTO']*ROTACJA_NALEŻNOŚCI*12)/365+(t_model['A. PRZYCHODY NETTO']*ROTACJA_ZAPASÓW*12)/365
t_model['1. Należności']=(t_model['A. PRZYCHODY NETTO']*ROTACJA_NALEŻNOŚCI*12)/365
t_model['2. Zapasy']=(t_model['A. PRZYCHODY NETTO']*ROTACJA_ZAPASÓW*12)/365
t_model['B. PASYWA BIEŻĄCE']=(-(t_model[' 2. Zużycie materiałów i energii']+t_model[' 3. Usługi obce']+t_model[' 7. Pozostałe koszty rodzajowe'])*ROTACJA_ZOBOWIĄZAŃ*12)/365
t_model['1. Zobowiązania']=(-(t_model[' 2. Zużycie materiałów i energii']+t_model[' 3. Usługi obce']+t_model[' 7. Pozostałe koszty rodzajowe'])*ROTACJA_ZOBOWIĄZAŃ*12)/365
t_model['C. KAPITAŁ OBROTOWY NETTO']=t_model['A. AKTYWA BIEŻĄCE']-t_model['B. PASYWA BIEŻĄCE']
t_model['temp']=t_model.shift(1)['C. KAPITAŁ OBROTOWY NETTO']
t_model['D. ZAPOTRZEBOWANIE NA KON']=-t_model.apply(lambda x:x['C. KAPITAŁ OBROTOWY NETTO'] if x['I']==0 else x['C. KAPITAŁ OBROTOWY NETTO']-x['temp'],axis=1)
t_model.drop(['temp'],axis=1,inplace=True)


In [250]:
# Przepływy pieniężne

t_model['A. WPŁYWY PIENIĘŻNE OGÓŁEM']=t_model.apply(lambda x:10000000 if x['I']==0 else 0, axis=1)+0+t_model['HAULER_CAPEX']+t_model['HAULER_DP']+t_model['DIGGER_CAPEX']+t_model['DIGGER_DP']+t_model['LOADER_CAPEX']+t_model['LOADER_DP']+0+t_model['A. PRZYCHODY NETTO']+t_model['C. POZOSTAŁE PRZYCHODY OPERACYJNE']+t_model['E. PRZYCHODY FINANSOWE']

t_model['1. Kapitał zakładowy']=t_model.apply(lambda x:10000000 if x['I']==0 else 0, axis=1)
t_model['2. Kredyty bankowe']=0
t_model['3. Leasing finansowy']=t_model['HAULER_CAPEX']+t_model['HAULER_DP']+t_model['DIGGER_CAPEX']+t_model['DIGGER_DP']*DIGGER_AMOUNT+t_model['LOADER_CAPEX']+t_model['LOADER_DP']*LOADER_AMOUNT
t_model['4. Inne']=0
t_model['5. Przychody ze sprzedaży']=t_model['A. PRZYCHODY NETTO']+t_model['C. POZOSTAŁE PRZYCHODY OPERACYJNE']+t_model['E. PRZYCHODY FINANSOWE']

t_model['B. WYDATKI PIENIĘŻNE OGÓŁEM']=-(t_model['HAULER_CAPEX']+t_model['DIGGER_CAPEX']+t_model['LOADER_CAPEX'])+t_model['B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ']-t_model[' 1. Amortyzacja']+t_model['D. POZOSTAŁE KOSZTY OPERACYJNE']+t_model['HAULER_PPMT']+t_model['DIGGER_PPMT']+t_model['LOADER_PPMT']+(t_model['HAULER_RV']+t_model['DIGGER_RV']+t_model['LOADER_RV'])+t_model['F. KOSZTY FINANSOWE']+t_model['H. PODATEK DOCHODOWY']

t_model['1. Nakłady inwestycyjne']=-(t_model['HAULER_CAPEX']+t_model['DIGGER_CAPEX']+t_model['LOADER_CAPEX'])
t_model['2. Koszty operacyjne (bez amortyzacji)']=t_model['B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ']-t_model[' 1. Amortyzacja']
t_model['3. Pozostałe koszty operacyjne']=t_model['D. POZOSTAŁE KOSZTY OPERACYJNE']
t_model['4. Spłaty kredytów i leasingów']=t_model['HAULER_PPMT']+t_model['DIGGER_PPMT']+t_model['LOADER_PPMT']+(t_model['HAULER_RV']+t_model['DIGGER_RV']+t_model['LOADER_RV'])
t_model['5. Odsetki']=t_model['F. KOSZTY FINANSOWE']
t_model['6. Podatek dochodowy']=t_model['H. PODATEK DOCHODOWY']

t_model['C. ZAPOTRZEBOWANIE NA KON']=t_model['D. ZAPOTRZEBOWANIE NA KON']

t_model['D. ZMIANA ŚRODKÓW PIENIĘŻNYCH']=t_model['A. WPŁYWY PIENIĘŻNE OGÓŁEM']+t_model['B. WYDATKI PIENIĘŻNE OGÓŁEM']+t_model['C. ZAPOTRZEBOWANIE NA KON']
t_model['E. SKUMULOWANE SALDO ŚP']=t_model['D. ZMIANA ŚRODKÓW PIENIĘŻNYCH'].cumsum()


In [251]:
# Bilans - Aktywa

t_model['A. AKTYWA TRWAŁE']=t_model['HAULER_CAPEX'].cumsum()+t_model['HAULER_DEPREC'].cumsum()+t_model['DIGGER_CAPEX'].cumsum()+t_model['DIGGER_DEPREC'].cumsum()+t_model['LOADER_CAPEX'].cumsum()+t_model['LOADER_DEPREC'].cumsum()

t_model['1. Wartości niematerialne i prawne']=0
t_model['2. Rzeczowe aktywa trwałe']=t_model['HAULER_CAPEX'].cumsum()+t_model['HAULER_DEPREC'].cumsum()+t_model['DIGGER_CAPEX'].cumsum()+t_model['DIGGER_DEPREC'].cumsum()+t_model['LOADER_CAPEX'].cumsum()+t_model['LOADER_DEPREC'].cumsum()
t_model['3. Należności długoterminowe']=0
t_model['4. Długoterminowe rozliczenia międzyokresowe']=0

t_model['B. AKTYWA OBROTOWE']=t_model['2. Zapasy']+t_model['1. Należności']+t_model['E. SKUMULOWANE SALDO ŚP']

t_model['1. Zapasy']=t_model['2. Zapasy']
t_model['2. Należności krótkoterminowe']=t_model['1. Należności']
t_model['3. Inwestycje krótkoterminowe']=t_model['E. SKUMULOWANE SALDO ŚP']
t_model['4. Krótkoterminowe rozliczenia międzyokresowe']=0

t_model['AKTYWA RAZEM']=t_model['A. AKTYWA TRWAŁE']+t_model['B. AKTYWA OBROTOWE']



In [252]:
# Bilans - Pasywa

t_model['A. KAPITAŁ WŁASNY']=t_model['1. Kapitał zakładowy'].cumsum()+t_model['I. WYNIK NETTO'].cumsum()
t_model['1. Kapitał podstawowy']=t_model['1. Kapitał zakładowy'].cumsum()
t_model['2. Kapitał zapasowy']=0
t_model['3. Kapitał z aktualizacji wyceny']=0
t_model['4. Pozostałe kapitały rezerwowe']=0
t_model['5. Zysk (strata) z lat ubiegłych']=0
t_model['6. Zysk (strata)']=t_model['I. WYNIK NETTO'].cumsum()
t_model['7. Odpisy w ciągu roku']=0


t_model['B. ZOBOWIĄZANIA I REZERWY']=t_model['HAULER_OUT']+t_model['DIGGER_OUT']+t_model['LOADER_OUT']+t_model['1. Zobowiązania']
t_model['1. Rezerwy na zobowiązania']=0
t_model['2. Zobowiązania długoterminowe']=t_model['HAULER_OUT']+t_model['DIGGER_OUT']+t_model['LOADER_OUT']
t_model['3. Zobowiązania krótkoterminowe']=t_model['1. Zobowiązania']
t_model['4. Rozliczenia międzyokresowe']=0



t_model['PASYWA RAZEM']=t_model['A. KAPITAŁ WŁASNY']+t_model['B. ZOBOWIĄZANIA I REZERWY']

t_model['Sprawdzenie']=t_model['AKTYWA RAZEM']-t_model['PASYWA RAZEM']


In [253]:
# CAPEX DIGGER

temp=t_model[['R',
'DIGGER_CAPEX',
 'DIGGER_DEPREC',
 'DIGGER_DP',
 'DIGGER_PPMT',
 'DIGGER_IPMT',
 'DIGGER_RV']].copy()

temp=temp.groupby(['R'],as_index=False)[['DIGGER_CAPEX',
 'DIGGER_DEPREC',
 'DIGGER_DP',
 'DIGGER_PPMT',
 'DIGGER_IPMT',
 'DIGGER_RV']].sum()

temp.rename(columns={'DIGGER_CAPEX':'Nakłady','DIGGER_DEPREC':'Amortyzacja','DIGGER_DP':'Opłata wstępna','DIGGER_PPMT':'Spłata kapitału','DIGGER_IPMT':'Spłata odsetek','DIGGER_RV':'Wykup'},inplace=True)
temp['Zaangażowanie leasingu']=temp['Nakłady']+temp['Opłata wstępna']+temp['Spłata kapitału']+temp['Wykup']
temp['Zaangażowanie leasingu']=temp['Zaangażowanie leasingu'].cumsum()
temp['Środek trwały']=temp['Nakłady']+temp['Amortyzacja']
temp['Środek trwały']=temp['Środek trwały'].cumsum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()
temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034])

# Tabela do raportu - wartości

tabela=temp[['POZYCJA',2023,2024,2025,2026]].copy()
tabela=tabela.style.format({2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'CAPEX_DIGGER_1.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2027,2028,2029,2030]].copy()
tabela=tabela.style.format({2027:'{0:,.2f}',2028:'{0:,.2f}',2029:'{0:,.2f}',2030:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'CAPEX_DIGGER_2.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2031,2032,2033,2034]].copy()
tabela=tabela.style.format({2031:'{0:,.2f}',2032:'{0:,.2f}',2033:'{0:,.2f}',2034:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'CAPEX_DIGGER_3.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

temp

,POZYCJA,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034
0,Nakłady,"1,783.69",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Amortyzacja,-137.21,-149.68,-149.68,-149.68,-149.68,-149.68,-149.68,-149.68,-149.68,-149.68,-149.68,-149.68
2,Opłata wstępna,-535.11,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,Spłata kapitału,-145.35,-173.23,-189.97,-208.33,-228.46,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Spłata odsetek,-100.46,-94.93,-78.19,-59.83,-39.70,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,Wykup,0.00,0.00,0.00,0.00,-303.23,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,Zaangażowanie leasingu,"1,103.23",929.99,740.02,531.69,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00
7,Środek trwały,"1,646.48","1,496.80","1,347.12","1,197.44","1,047.76",898.08,748.40,598.72,449.04,299.36,149.68,0.00


In [254]:
# CAPEX HAULER

temp=t_model[['R',
'HAULER_CAPEX',
 'HAULER_DEPREC',
 'HAULER_DP',
 'HAULER_PPMT',
 'HAULER_IPMT',
 'HAULER_RV']].copy()

temp=temp.groupby(['R'],as_index=False)[['HAULER_CAPEX',
 'HAULER_DEPREC',
 'HAULER_DP',
 'HAULER_PPMT',
 'HAULER_IPMT',
 'HAULER_RV']].sum()

temp.rename(columns={'HAULER_CAPEX':'Nakłady','HAULER_DEPREC':'Amortyzacja','HAULER_DP':'Opłata wstępna','HAULER_PPMT':'Spłata kapitału','HAULER_IPMT':'Spłata odsetek','HAULER_RV':'Wykup'},inplace=True)
temp['Zaangażowanie leasingu']=temp['Nakłady']+temp['Opłata wstępna']+temp['Spłata kapitału']+temp['Wykup']
temp['Zaangażowanie leasingu']=temp['Zaangażowanie leasingu'].cumsum()
temp['Środek trwały']=temp['Nakłady']+temp['Amortyzacja']
temp['Środek trwały']=temp['Środek trwały'].cumsum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()
temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034])

# Tabela do raportu - wartości

tabela=temp[['POZYCJA',2023,2024,2025,2026]].copy()
tabela=tabela.style.format({2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'CAPEX_HAULER_1.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2027,2028,2029,2030]].copy()
tabela=tabela.style.format({2027:'{0:,.2f}',2028:'{0:,.2f}',2029:'{0:,.2f}',2030:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'CAPEX_HAULER_2.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2031,2032,2033,2034]].copy()
tabela=tabela.style.format({2031:'{0:,.2f}',2032:'{0:,.2f}',2033:'{0:,.2f}',2034:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'CAPEX_HAULER_3.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

temp

,POZYCJA,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034
0,Nakłady,"4,602.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Amortyzacja,-354.00,-386.18,-386.18,-386.18,-386.18,-386.18,-386.18,-386.18,-386.18,-386.18,-386.18,-386.18
2,Opłata wstępna,"-1,380.60",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,Spłata kapitału,-332.60,-396.40,-434.70,-476.71,-522.77,-45.78,0.00,0.00,0.00,0.00,0.00,0.00
4,Spłata odsetek,-260.81,-250.96,-212.65,-170.65,-124.58,-8.17,0.00,0.00,0.00,0.00,0.00,0.00
5,Wykup,0.00,0.00,0.00,0.00,0.00,"-1,012.44",0.00,0.00,0.00,0.00,0.00,0.00
6,Zaangażowanie leasingu,"2,888.80","2,492.40","2,057.70","1,580.99","1,058.22",-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00
7,Środek trwały,"4,248.00","3,861.82","3,475.64","3,089.45","2,703.27","2,317.09","1,930.91","1,544.73","1,158.55",772.36,386.18,-0.00


In [255]:

temp=t_model[['R',
'LOADER_CAPEX',
 'LOADER_DEPREC',
 'LOADER_DP',
 'LOADER_PPMT',
 'LOADER_IPMT',
 'LOADER_RV']].copy()

temp=temp.groupby(['R'],as_index=False)[['LOADER_CAPEX',
 'LOADER_DEPREC',
 'LOADER_DP',
 'LOADER_PPMT',
 'LOADER_IPMT',
 'LOADER_RV']].sum()

temp.rename(columns={'LOADER_CAPEX':'Nakłady','LOADER_DEPREC':'Amortyzacja','LOADER_DP':'Opłata wstępna','LOADER_PPMT':'Spłata kapitału','LOADER_IPMT':'Spłata odsetek','LOADER_RV':'Wykup'},inplace=True)
temp['Zaangażowanie leasingu']=temp['Nakłady']+temp['Opłata wstępna']+temp['Spłata kapitału']+temp['Wykup']
temp['Zaangażowanie leasingu']=temp['Zaangażowanie leasingu'].cumsum()
temp['Środek trwały']=temp['Nakłady']+temp['Amortyzacja']
temp['Środek trwały']=temp['Środek trwały'].cumsum()

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()
temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034])

# Tabela do raportu - wartości

tabela=temp[['POZYCJA',2023,2024,2025,2026]].copy()
tabela=tabela.style.format({2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'CAPEX_LOADER_1.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2027,2028,2029,2030]].copy()
tabela=tabela.style.format({2027:'{0:,.2f}',2028:'{0:,.2f}',2029:'{0:,.2f}',2030:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'CAPEX_LOADER_2.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2031,2032,2033,2034]].copy()
tabela=tabela.style.format({2031:'{0:,.2f}',2032:'{0:,.2f}',2033:'{0:,.2f}',2034:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'CAPEX_LOADER_3.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

temp

,POZYCJA,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034
0,Nakłady,"1,814.84",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Amortyzacja,-139.60,-152.29,-152.29,-152.29,-152.29,-152.29,-152.29,-152.29,-152.29,-152.29,-152.29,-152.29
2,Opłata wstępna,-544.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,Spłata kapitału,-142.31,-169.61,-186.00,-203.97,-223.68,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Spłata odsetek,-102.43,-97.38,-80.99,-63.02,-43.31,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,Wykup,0.00,0.00,0.00,0.00,-344.82,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,Zaangażowanie leasingu,"1,128.08",958.47,772.47,568.50,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00
7,Środek trwały,"1,675.24","1,522.94","1,370.65","1,218.35","1,066.06",913.77,761.47,609.18,456.88,304.59,152.29,-0.00


In [256]:
temp = t_model[['R','PROD_MONTH','PROD_SHIFT','DIGGER_SHIFT_TIME','HAULER_SHIFT_TIME','HAULER_ADD_TIME','DIGGER_ADD_TIME','INCOME_SHIFT','INCOME_ADD_TIME','INCOME_LOADER']].copy()

temp=temp.groupby(['R'],as_index=False).agg(
    Produkcja=('PROD_MONTH','sum'),
    Max_czas_pracy_koparki=('DIGGER_SHIFT_TIME','max'),
    Min_czas_pracy_koparki=('DIGGER_SHIFT_TIME','min'),
    Max_czas_pracy_wozidła=('HAULER_SHIFT_TIME','max'),
    Min_czas_pracy_wozidła=('HAULER_SHIFT_TIME','min'),
    Add_czas_pracy_wozidła=('HAULER_ADD_TIME','mean'),
    Add_czas_pracy_koparki=('DIGGER_ADD_TIME','mean'),
    Przychód_Zmiana=('INCOME_SHIFT','sum'),
    Przychód_Czas=('INCOME_ADD_TIME','sum'),
    Przychód_Ład=('INCOME_LOADER','sum')
    )

temp.rename(columns={'Produkcja':'Produkcja roczna','Max_czas_pracy_koparki':'Maksymalny czas pracy koparki na zmianie', 'Min_czas_pracy_koparki':'Minimalny czas pracy koparki na zmianie',
       'Max_czas_pracy_wozidła':'Maksymalny czas pracy wozidła na zmianie', 'Min_czas_pracy_wozidła':'Minimalny czas pracy wozidła na zmianie','Add_czas_pracy_wozidła':'Dodatkowy czas pracy wozidła', 'Add_czas_pracy_koparki':'Dodatkowy czas pracy koparki', 'Przychód_Zmiana':'Przychód z załadunku i transportu','Przychód_Czas':'Dodatkowy przychód z wynajmu', 'Przychód_Ład':'Przychód z wynajmu ładowarki'},inplace=True)

temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp['Produkcja roczna']=temp['Produkcja roczna']/1000
temp['Przychód z załadunku i transportu']=temp['Przychód z załadunku i transportu']/1000
temp['Dodatkowy przychód z wynajmu']=temp['Dodatkowy przychód z wynajmu']/1000
temp['Przychód z wynajmu ładowarki']=temp['Przychód z wynajmu ładowarki']/1000
temp=temp.T
temp=temp.reset_index()
temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034])

# Tabela do raportu - wartości

tabela=temp[['POZYCJA',2023,2024,2025,2026]].copy()
tabela=tabela.style.format({2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'INCOME_1.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2027,2028,2029,2030]].copy()
tabela=tabela.style.format({2027:'{0:,.2f}',2028:'{0:,.2f}',2029:'{0:,.2f}',2030:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'INCOME_2.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2031,2032,2033,2034]].copy()
tabela=tabela.style.format({2031:'{0:,.2f}',2032:'{0:,.2f}',2033:'{0:,.2f}',2034:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'INCOME_3.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

temp


,POZYCJA,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034
0,Produkcja roczna,900.00,918.00,936.36,955.09,974.19,993.67,"1,013.55","1,033.82","1,054.49","1,075.58","1,097.09","1,119.04"
1,Maksymalny czas pracy koparki na zmianie,3.66,3.74,3.81,3.89,3.97,4.04,4.13,4.21,4.29,4.38,4.47,4.56
2,Minimalny czas pracy koparki na zmianie,1.13,1.15,1.17,1.20,1.22,1.24,1.27,1.29,1.32,1.35,1.37,1.40
3,Maksymalny czas pracy wozidła na zmianie,4.73,4.82,4.92,5.02,5.12,5.22,5.32,5.43,5.54,5.65,5.76,5.88
4,Minimalny czas pracy wozidła na zmianie,1.45,1.48,1.51,1.54,1.57,1.61,1.64,1.67,1.70,1.74,1.77,1.81
5,Dodatkowy czas pracy wozidła,250.00,250.00,250.00,250.00,250.00,250.00,250.00,250.00,250.00,250.00,250.00,250.00
6,Dodatkowy czas pracy koparki,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
7,Przychód z załadunku i transportu,"3,591.00","3,662.82","3,736.08","3,810.80","3,887.01","3,964.75","4,044.05","4,124.93","4,207.43","4,291.58","4,377.41","4,464.96"
8,Dodatkowy przychód z wynajmu,"1,128.00","1,128.00","1,128.00","1,128.00","1,128.00","1,128.00","1,128.00","1,128.00","1,128.00","1,128.00","1,128.00","1,128.00"
9,Przychód z wynajmu ładowarki,"1,821.05","1,711.15","1,848.05","1,711.15","1,711.15","1,557.03","1,420.13","1,557.03","1,420.13","1,420.13","1,557.03","1,420.13"


In [257]:

# COSTS HAULER

temp=t_model[['R',
'HAULER_INS', 
'HAULER_TNMR_COST',
'HAULER_FUEL_COST',
'HAULER_TYRES_COST',
#'HAULER_BUCKET_COST',
'HAULER_SOUNDRY_COST']].copy()

temp=temp.groupby(['R'],as_index=False)[[
    'HAULER_INS',
    'HAULER_TNMR_COST',
    'HAULER_FUEL_COST',
    'HAULER_TYRES_COST',
    #'HAULER_BUCKET_COST',
    'HAULER_SOUNDRY_COST']].sum()

temp.rename(columns={'HAULER_INS':'Koszt ubezpieczenia',
    'HAULER_TNMR_COST':'Koszt serwisowy',
    'HAULER_FUEL_COST':'Koszt zużycia paliwa',
    'HAULER_TYRES_COST':'Koszt wymiany opon',
    #'HAULER_BUCKET_COST':'Koszt serwisu lemieszy i łyżek',
    'HAULER_SOUNDRY_COST':'Koszty pozostałe'},inplace=True)


temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()
temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034])

# Tabela do raportu - wartości

tabela=temp[['POZYCJA',2023,2024,2025,2026]].copy()
tabela=tabela.style.format({2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'COST_HAULER_1.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2027,2028,2029,2030]].copy()
tabela=tabela.style.format({2027:'{0:,.2f}',2028:'{0:,.2f}',2029:'{0:,.2f}',2030:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'COST_HAULER_2.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2031,2032,2033,2034]].copy()
tabela=tabela.style.format({2031:'{0:,.2f}',2032:'{0:,.2f}',2033:'{0:,.2f}',2034:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'COST_HAULER_3.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

temp

,POZYCJA,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034
0,Koszt ubezpieczenia,-52.95,-57.76,-57.76,-57.76,-57.76,-57.76,-57.76,-57.76,-57.76,-57.76,-57.76,-57.76
1,Koszt serwisowy,-362.74,-367.72,-372.79,-377.97,-383.25,-388.63,-394.13,-399.73,-405.44,-411.27,-417.22,-423.28
2,Koszt zużycia paliwa,-921.96,-934.60,-947.50,-960.65,-974.07,-987.76,"-1,001.72","-1,015.96","-1,030.48","-1,045.30","-1,060.41","-1,075.82"
3,Koszt wymiany opon,-63.20,0.00,-63.20,0.00,0.00,-63.20,0.00,-63.20,0.00,0.00,-63.20,0.00
4,Koszty pozostałe,-18.15,-19.80,-19.80,-19.80,-19.80,-19.80,-19.80,-19.80,-19.80,-19.80,-19.80,-19.80


In [258]:
# COSTS LOADER

temp=t_model[['R',
'LOADER_INS', 
'LOADER_TNMR_COST',
'LOADER_FUEL_COST',
'LOADER_TYRES_COST',
'LOADER_BUCKET_COST',
'LOADER_SOUNDRY_COST']].copy()

temp=temp.groupby(['R'],as_index=False)[[
    'LOADER_INS',
    'LOADER_TNMR_COST',
    'LOADER_FUEL_COST',
    'LOADER_TYRES_COST',
    'LOADER_BUCKET_COST',
    'LOADER_SOUNDRY_COST']].sum()

temp.rename(columns={'LOADER_INS':'Koszt ubezpieczenia',
    'LOADER_TNMR_COST':'Koszt serwisowy',
    'LOADER_FUEL_COST':'Koszt zużycia paliwa',
    'LOADER_TYRES_COST':'Koszt wymiany opon',
    'LOADER_BUCKET_COST':'Koszt serwisu lemieszy i łyżek',
    'LOADER_SOUNDRY_COST':'Koszty pozostałe'},inplace=True)


temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()
temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034])

# Tabela do raportu - wartości

tabela=temp[['POZYCJA',2023,2024,2025,2026]].copy()
tabela=tabela.style.format({2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'COST_LOADER_1.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2027,2028,2029,2030]].copy()
tabela=tabela.style.format({2027:'{0:,.2f}',2028:'{0:,.2f}',2029:'{0:,.2f}',2030:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'COST_LOADER_2.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2031,2032,2033,2034]].copy()
tabela=tabela.style.format({2031:'{0:,.2f}',2032:'{0:,.2f}',2033:'{0:,.2f}',2034:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'COST_LOADER_3.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

temp

,POZYCJA,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034
0,Koszt ubezpieczenia,-20.58,-22.45,-22.45,-22.45,-22.45,-22.45,-22.45,-22.45,-22.45,-22.45,-22.45,-22.45
1,Koszt serwisowy,-133.20,-133.20,-133.20,-133.20,-133.20,-133.20,-133.20,-133.20,-133.20,-133.20,-133.20,-133.20
2,Koszt zużycia paliwa,"-1,089.42","-1,089.42","-1,089.42","-1,089.42","-1,089.42","-1,089.42","-1,089.42","-1,089.42","-1,089.42","-1,089.42","-1,089.42","-1,089.42"
3,Koszt wymiany opon,-125.60,0.00,-125.60,0.00,0.00,-125.60,0.00,-125.60,0.00,0.00,-125.60,0.00
4,Koszt serwisu lemieszy i łyżek,-50.00,-50.00,-50.00,-50.00,-50.00,-50.00,-50.00,-50.00,-50.00,-50.00,-50.00,-50.00
5,Koszty pozostałe,-7.15,-7.80,-7.80,-7.80,-7.80,-7.80,-7.80,-7.80,-7.80,-7.80,-7.80,-7.80


In [259]:
# COSTS DIGGER

temp=t_model[['R',
'DIGGER_INS', 
'DIGGER_TNMR_COST',
'DIGGER_FUEL_COST',
#'DIGGER_TYRES_COST',
'DIGGER_BUCKET_COST',
'DIGGER_SOUNDRY_COST']].copy()

temp=temp.groupby(['R'],as_index=False)[[
    'DIGGER_INS',
    'DIGGER_TNMR_COST',
    'DIGGER_FUEL_COST',
    #'DIGGER_TYRES_COST',
    'DIGGER_BUCKET_COST',
    'DIGGER_SOUNDRY_COST']].sum()

temp.rename(columns={'DIGGER_INS':'Koszt ubezpieczenia',
    'DIGGER_TNMR_COST':'Koszt serwisowy',
    'DIGGER_FUEL_COST':'Koszt zużycia paliwa',
    #'DIGGER_TYRES_COST':'Koszt wymiany opon',
    'DIGGER_BUCKET_COST':'Koszt serwisu lemieszy i łyżek',
    'DIGGER_SOUNDRY_COST':'Koszty pozostałe'},inplace=True)


temp.rename(columns={'R':'POZYCJA'},inplace=True)
temp=temp.set_index('POZYCJA')
temp=temp/1000
temp=temp.T
temp=temp.reset_index()
temp=pd.DataFrame(temp.values.tolist(),columns=['POZYCJA',2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034])

# Tabela do raportu - wartości

tabela=temp[['POZYCJA',2023,2024,2025,2026]].copy()
tabela=tabela.style.format({2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'COST_DIGGER_1.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2027,2028,2029,2030]].copy()
tabela=tabela.style.format({2027:'{0:,.2f}',2028:'{0:,.2f}',2029:'{0:,.2f}',2030:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'COST_DIGGER_2.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2031,2032,2033,2034]].copy()
tabela=tabela.style.format({2031:'{0:,.2f}',2032:'{0:,.2f}',2033:'{0:,.2f}',2034:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'COST_DIGGER_3.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

temp

,POZYCJA,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034
0,Koszt ubezpieczenia,-20.27,-22.11,-22.11,-22.11,-22.11,-22.11,-22.11,-22.11,-22.11,-22.11,-22.11,-22.11
1,Koszt serwisowy,-104.07,-105.29,-106.53,-107.80,-109.09,-110.40,-111.75,-113.12,-114.52,-115.94,-117.40,-118.88
2,Koszt zużycia paliwa,-874.81,-885.05,-895.49,-906.13,-916.99,-928.07,-939.37,-950.89,-962.65,-974.64,-986.87,-999.34
3,Koszt serwisu lemieszy i łyżek,-23.20,-23.20,-23.20,-23.20,-23.20,-23.20,-23.20,-23.20,-23.20,-23.20,-23.20,-23.20
4,Koszty pozostałe,-13.75,-15.00,-15.00,-15.00,-15.00,-15.00,-15.00,-15.00,-15.00,-15.00,-15.00,-15.00


In [260]:
# Rachunek wyników - tabela

t_pl=t_model[['R','A. PRZYCHODY NETTO',
       ' 1. Przychody ze sprzedaży usług',
       ' 2. Przychody dodatkowe ze sprzedaży usług', ' 3. Przychody z wynajmu',
       'B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ', ' 1. Amortyzacja',
       ' 2. Zużycie materiałów i energii', ' 3. Usługi obce',
       ' 4. Podatki i opłaty', ' 5. Wynagrodzenia',
       ' 6. Ubezpieczenia społeczne', ' 7. Pozostałe koszty rodzajowe',
       ' 8. Wartość sprzedanych towarów i materiałów',
       'C. POZOSTAŁE PRZYCHODY OPERACYJNE', 'D. POZOSTAŁE KOSZTY OPERACYJNE',
       'E. PRZYCHODY FINANSOWE', 'F. KOSZTY FINANSOWE', 'G. WYNIK BRUTTO',
       'H. PODATEK DOCHODOWY', 'I. WYNIK NETTO']].copy()

t_pl=t_pl.groupby(['R'],as_index=False)[['A. PRZYCHODY NETTO',
       ' 1. Przychody ze sprzedaży usług',
       ' 2. Przychody dodatkowe ze sprzedaży usług', ' 3. Przychody z wynajmu',
       'B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ', ' 1. Amortyzacja',
       ' 2. Zużycie materiałów i energii', ' 3. Usługi obce',
       ' 4. Podatki i opłaty', ' 5. Wynagrodzenia',
       ' 6. Ubezpieczenia społeczne', ' 7. Pozostałe koszty rodzajowe',
       ' 8. Wartość sprzedanych towarów i materiałów',
       'C. POZOSTAŁE PRZYCHODY OPERACYJNE', 'D. POZOSTAŁE KOSZTY OPERACYJNE',
       'E. PRZYCHODY FINANSOWE', 'F. KOSZTY FINANSOWE', 'G. WYNIK BRUTTO',
       'H. PODATEK DOCHODOWY', 'I. WYNIK NETTO']].sum()

t_pl.rename(columns={'R':'POZYCJA'},inplace=True)
t_pl=t_pl.set_index('POZYCJA')
t_pl=t_pl/1000
t_pl=t_pl.T
t_pl=t_pl.reset_index()
t_pl=pd.DataFrame(t_pl.values.tolist(),columns=['POZYCJA',2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034])
t_pl



,POZYCJA,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034
0,A. PRZYCHODY NETTO,"6,540.05","6,501.97","6,712.13","6,649.94","6,726.16","6,649.79","6,592.18","6,809.96","6,755.56","6,839.71","7,062.44","7,013.09"
1,1. Przychody ze sprzedaży usług,"3,591.00","3,662.82","3,736.08","3,810.80","3,887.01","3,964.75","4,044.05","4,124.93","4,207.43","4,291.58","4,377.41","4,464.96"
2,2. Przychody dodatkowe ze sprzedaży usług,"1,128.00","1,128.00","1,128.00","1,128.00","1,128.00","1,128.00","1,128.00","1,128.00","1,128.00","1,128.00","1,128.00","1,128.00"
3,3. Przychody z wynajmu,"1,821.05","1,711.15","1,848.05","1,711.15","1,711.15","1,557.03","1,420.13","1,557.03","1,420.13","1,420.13","1,557.03","1,420.13"
4,B. KOSZTY DZIAŁALNOŚCI OPERACYJNEJ,"-5,776.46","-5,801.12","-6,019.57","-5,861.02","-5,891.87","-6,112.13","-5,955.43","-6,176.97","-6,021.56","-6,055.62","-6,279.16","-6,125.79"
5,1. Amortyzacja,-630.81,-688.16,-688.16,-688.16,-688.16,-688.16,-688.16,-688.16,-688.16,-688.16,-688.16,-688.16
6,2. Zużycie materiałów i energii,"-3,148.20","-2,982.27","-3,194.41","-3,029.41","-3,053.69","-3,267.25","-3,103.71","-3,318.28","-3,155.76","-3,182.56","-3,398.70","-3,237.79"
7,3. Usługi obce,-693.80,-708.52,-714.84,-721.28,-727.85,-734.55,-741.39,-748.36,-755.48,-762.73,-770.13,-777.68
8,4. Podatki i opłaty,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,5. Wynagrodzenia,"-1,028.72","-1,122.24","-1,122.24","-1,122.24","-1,122.24","-1,122.24","-1,122.24","-1,122.24","-1,122.24","-1,122.24","-1,122.24","-1,122.24"


In [261]:
# Tabela do raportu - wartości
temp=t_pl

tabela=temp[['POZYCJA',2023,2024,2025,2026]].copy()
tabela=tabela.style.format({2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'RW_1.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2027,2028,2029,2030]].copy()
tabela=tabela.style.format({2027:'{0:,.2f}',2028:'{0:,.2f}',2029:'{0:,.2f}',2030:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'RW_2.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2031,2032,2033,2034]].copy()
tabela=tabela.style.format({2031:'{0:,.2f}',2032:'{0:,.2f}',2033:'{0:,.2f}',2034:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'RW_3.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')


In [262]:
# Zapotrzebowanie ma KO Tabela

t_kon=t_model[['R','A. AKTYWA BIEŻĄCE', '1. Należności', '2. Zapasy', 'B. PASYWA BIEŻĄCE', '1. Zobowiązania']].copy()
t_kon=t_kon.groupby(['R'],as_index=False)[['A. AKTYWA BIEŻĄCE', '1. Należności', '2. Zapasy', 'B. PASYWA BIEŻĄCE', '1. Zobowiązania']].last()
t_kon['C. KAPITAŁ OBROTOWY NETTO']=t_kon['A. AKTYWA BIEŻĄCE']-t_kon['B. PASYWA BIEŻĄCE']
t_kon['temp']=t_kon.shift(1)['C. KAPITAŁ OBROTOWY NETTO']
t_kon['D. ZAPOTRZEBOWANIE NA KON']=-t_kon.apply(lambda x:x['C. KAPITAŁ OBROTOWY NETTO'] if x['R']==DATE_START.year else x['C. KAPITAŁ OBROTOWY NETTO']-x['temp'],axis=1)
t_kon.drop(['temp'],axis=1,inplace=True)

t_kon.rename(columns={'R':'POZYCJA'},inplace=True)
t_kon=t_kon.set_index('POZYCJA')

t_kon=t_kon/1000
t_kon=t_kon.T
t_kon=t_kon.reset_index()
t_kon=pd.DataFrame(t_kon.values.tolist(),columns=['POZYCJA',2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034])
t_kon


,POZYCJA,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034
0,A. AKTYWA BIEŻĄCE,198.01,199.33,200.68,202.05,203.46,193.73,195.19,196.67,198.19,199.74,201.32,202.94
1,1. Należności,198.01,199.33,200.68,202.05,203.46,193.73,195.19,196.67,198.19,199.74,201.32,202.94
2,2. Zapasy,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,B. PASYWA BIEŻĄCE,308.52,309.67,310.84,312.03,313.25,314.49,315.76,317.05,318.37,319.71,321.08,322.48
4,1. Zobowiązania,308.52,309.67,310.84,312.03,313.25,314.49,315.76,317.05,318.37,319.71,321.08,322.48
5,C. KAPITAŁ OBROTOWY NETTO,-110.52,-110.34,-110.16,-109.98,-109.79,-120.77,-120.57,-120.37,-120.17,-119.97,-119.76,-119.54
6,D. ZAPOTRZEBOWANIE NA KON,110.52,-0.18,-0.18,-0.18,-0.19,10.97,-0.19,-0.20,-0.20,-0.21,-0.21,-0.21


In [263]:
# Tabela do raportu - wartości
temp=t_kon

tabela=temp[['POZYCJA',2023,2024,2025,2026]].copy()
tabela=tabela.style.format({2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'KON_1.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2027,2028,2029,2030]].copy()
tabela=tabela.style.format({2027:'{0:,.2f}',2028:'{0:,.2f}',2029:'{0:,.2f}',2030:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'KON_2.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2031,2032,2033,2034]].copy()
tabela=tabela.style.format({2031:'{0:,.2f}',2032:'{0:,.2f}',2033:'{0:,.2f}',2034:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'KON_3.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

In [264]:
# Cash Flow - Tabela


t_cf=t_model[['R','A. WPŁYWY PIENIĘŻNE OGÓŁEM', '1. Kapitał zakładowy',
       '2. Kredyty bankowe', '3. Leasing finansowy', '4. Inne',
       '5. Przychody ze sprzedaży', 'B. WYDATKI PIENIĘŻNE OGÓŁEM',
       '1. Nakłady inwestycyjne', '2. Koszty operacyjne (bez amortyzacji)',
       '3. Pozostałe koszty operacyjne', '4. Spłaty kredytów i leasingów',
       '5. Odsetki', '6. Podatek dochodowy', 'C. ZAPOTRZEBOWANIE NA KON',
       'D. ZMIANA ŚRODKÓW PIENIĘŻNYCH']]

t_cf=t_cf.groupby(['R'],as_index=False)[['A. WPŁYWY PIENIĘŻNE OGÓŁEM', '1. Kapitał zakładowy',
       '2. Kredyty bankowe', '3. Leasing finansowy', '4. Inne',
       '5. Przychody ze sprzedaży', 'B. WYDATKI PIENIĘŻNE OGÓŁEM',
       '1. Nakłady inwestycyjne', '2. Koszty operacyjne (bez amortyzacji)',
       '3. Pozostałe koszty operacyjne', '4. Spłaty kredytów i leasingów',
       '5. Odsetki', '6. Podatek dochodowy', 'C. ZAPOTRZEBOWANIE NA KON',
       'D. ZMIANA ŚRODKÓW PIENIĘŻNYCH']].sum()

t_cf['E. SKUMULOWANE SALDO ŚP']=t_cf['D. ZMIANA ŚRODKÓW PIENIĘŻNYCH'].cumsum()
t_cf.rename(columns={'R':'POZYCJA'},inplace=True)
t_cf=t_cf.set_index('POZYCJA')

t_cf=t_cf/1000
t_cf=t_cf.T
t_cf=t_cf.reset_index()
t_cf=pd.DataFrame(t_cf.values.tolist(),columns=['POZYCJA',2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034])
t_cf

,POZYCJA,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034
0,A. WPŁYWY PIENIĘŻNE OGÓŁEM,"22,280.42","6,501.97","6,712.13","6,649.94","6,726.16","6,649.79","6,592.18","6,809.96","6,755.56","6,839.71","7,062.44","7,013.09"
1,1. Kapitał zakładowy,"10,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2. Kredyty bankowe,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,3. Leasing finansowy,"5,740.37",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,4. Inne,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,5. Przychody ze sprzedaży,"6,540.05","6,501.97","6,712.13","6,649.94","6,726.16","6,649.79","6,592.18","6,809.96","6,755.56","6,839.71","7,062.44","7,013.09"
6,B. WYDATKI PIENIĘŻNE OGÓŁEM,"-14,430.15","-6,295.47","-6,513.92","-6,355.37","-7,034.26","-6,490.36","-5,267.27","-5,488.81","-5,333.40","-5,367.46","-5,591.00","-5,437.64"
7,1. Nakłady inwestycyjne,"-8,200.53",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,2. Koszty operacyjne (bez amortyzacji),"-5,145.65","-5,112.96","-5,331.42","-5,172.86","-5,203.71","-5,423.98","-5,267.27","-5,488.81","-5,333.40","-5,367.46","-5,591.00","-5,437.64"
9,3. Pozostałe koszty operacyjne,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [265]:
# Tabela do raportu - wartości
temp=t_cf

tabela=temp[['POZYCJA',2023,2024,2025,2026]].copy()
tabela=tabela.style.format({2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'CF_1.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2027,2028,2029,2030]].copy()
tabela=tabela.style.format({2027:'{0:,.2f}',2028:'{0:,.2f}',2029:'{0:,.2f}',2030:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'CF_2.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2031,2032,2033,2034]].copy()
tabela=tabela.style.format({2031:'{0:,.2f}',2032:'{0:,.2f}',2033:'{0:,.2f}',2034:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'CF_3.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

In [266]:
# Bilans aktywa - Tabela


t_bil_a=t_model[['R','A. AKTYWA TRWAŁE',
 '1. Wartości niematerialne i prawne',
 '2. Rzeczowe aktywa trwałe',
 '3. Należności długoterminowe',
 '4. Długoterminowe rozliczenia międzyokresowe',
 'B. AKTYWA OBROTOWE',
 '1. Zapasy',
 '2. Należności krótkoterminowe',
 '3. Inwestycje krótkoterminowe',
 '4. Krótkoterminowe rozliczenia międzyokresowe',
 'AKTYWA RAZEM']].copy()
t_bil_a=t_bil_a.groupby(['R'],as_index=False)[['A. AKTYWA TRWAŁE',
 '1. Wartości niematerialne i prawne',
 '2. Rzeczowe aktywa trwałe',
 '3. Należności długoterminowe',
 '4. Długoterminowe rozliczenia międzyokresowe',
 'B. AKTYWA OBROTOWE',
 '1. Zapasy',
 '2. Należności krótkoterminowe',
 '3. Inwestycje krótkoterminowe',
 '4. Krótkoterminowe rozliczenia międzyokresowe',
 'AKTYWA RAZEM']].last()


t_bil_a.rename(columns={'R':'POZYCJA'},inplace=True)
t_bil_a=t_bil_a.set_index('POZYCJA')

t_bil_a=t_bil_a/1000
t_bil_a=t_bil_a.T
t_bil_a=t_bil_a.reset_index()
t_bil_a=pd.DataFrame(t_bil_a.values.tolist(),columns=['POZYCJA',2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034])
t_bil_a



,POZYCJA,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034
0,A. AKTYWA TRWAŁE,"7,569.72","6,881.56","6,193.41","5,505.25","4,817.09","4,128.94","3,440.78","2,752.62","2,064.47","1,376.31",688.16,-0.00
1,1. Wartości niematerialne i prawne,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2. Rzeczowe aktywa trwałe,"7,569.72","6,881.56","6,193.41","5,505.25","4,817.09","4,128.94","3,440.78","2,752.62","2,064.47","1,376.31",688.16,-0.00
3,3. Należności długoterminowe,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,4. Długoterminowe rozliczenia międzyokresowe,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,B. AKTYWA OBROTOWE,"8,158.80","8,366.44","8,565.82","8,861.59","8,554.70","8,715.37","10,041.54","11,363.99","12,787.46","14,261.05","15,733.86","17,310.70"
6,1. Zapasy,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,2. Należności krótkoterminowe,198.01,199.33,200.68,202.05,203.46,193.73,195.19,196.67,198.19,199.74,201.32,202.94
8,3. Inwestycje krótkoterminowe,"7,960.79","8,167.11","8,365.14","8,659.53","8,351.24","8,521.64","9,846.35","11,167.31","12,589.26","14,061.30","15,532.53","17,107.77"
9,4. Krótkoterminowe rozliczenia międzyokresowe,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [267]:
# Tabela do raportu - wartości
temp=t_bil_a

tabela=temp[['POZYCJA',2023,2024,2025,2026]].copy()
tabela=tabela.style.format({2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'BA_1.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2027,2028,2029,2030]].copy()
tabela=tabela.style.format({2027:'{0:,.2f}',2028:'{0:,.2f}',2029:'{0:,.2f}',2030:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'BA_2.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2031,2032,2033,2034]].copy()
tabela=tabela.style.format({2031:'{0:,.2f}',2032:'{0:,.2f}',2033:'{0:,.2f}',2034:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'BA_3.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

In [268]:
# Bilans Pasywa - tabela


t_bil_p=t_model[['R','A. KAPITAŁ WŁASNY',
 '1. Kapitał podstawowy',
 '2. Kapitał zapasowy',
 '3. Kapitał z aktualizacji wyceny',
 '4. Pozostałe kapitały rezerwowe',
 '5. Zysk (strata) z lat ubiegłych',
 '6. Zysk (strata)',
 '7. Odpisy w ciągu roku',
 'B. ZOBOWIĄZANIA I REZERWY',
 '1. Rezerwy na zobowiązania',
 '2. Zobowiązania długoterminowe',
 '3. Zobowiązania krótkoterminowe',
 '4. Rozliczenia międzyokresowe',
 'PASYWA RAZEM']].copy()
t_bil_p=t_bil_p.groupby(['R'],as_index=False)[['A. KAPITAŁ WŁASNY',
 '1. Kapitał podstawowy',
 '2. Kapitał zapasowy',
 '3. Kapitał z aktualizacji wyceny',
 '4. Pozostałe kapitały rezerwowe',
 '5. Zysk (strata) z lat ubiegłych',
 '6. Zysk (strata)',
 '7. Odpisy w ciągu roku',
 'B. ZOBOWIĄZANIA I REZERWY',
 '1. Rezerwy na zobowiązania',
 '2. Zobowiązania długoterminowe',
 '3. Zobowiązania krótkoterminowe',
 '4. Rozliczenia międzyokresowe',
 'PASYWA RAZEM']].last()


t_bil_p.rename(columns={'R':'POZYCJA'},inplace=True)
t_bil_p=t_bil_p.set_index('POZYCJA')

t_bil_p=t_bil_p/1000
t_bil_p=t_bil_p.T
t_bil_p=t_bil_p.reset_index()
t_bil_p=pd.DataFrame(t_bil_p.values.tolist(),columns=['POZYCJA',2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034])
t_bil_p


,POZYCJA,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034
0,A. KAPITAŁ WŁASNY,"10,299.89","10,557.47","10,878.19","11,373.62","12,000.32","12,529.81","13,166.56","13,799.56","14,533.56","15,317.65","16,100.93","16,988.22"
1,1. Kapitał podstawowy,"10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00"
2,2. Kapitał zapasowy,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,3. Kapitał z aktualizacji wyceny,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,4. Pozostałe kapitały rezerwowe,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,5. Zysk (strata) z lat ubiegłych,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,6. Zysk (strata),299.89,557.47,878.19,"1,373.62","2,000.32","2,529.81","3,166.56","3,799.56","4,533.56","5,317.65","6,100.93","6,988.22"
7,7. Odpisy w ciągu roku,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,B. ZOBOWIĄZANIA I REZERWY,"5,428.63","4,690.54","3,881.03","2,993.22","1,371.47",314.49,315.76,317.05,318.37,319.71,321.08,322.48
9,1. Rezerwy na zobowiązania,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [269]:
# Tabela do raportu - wartości
temp=t_bil_p

tabela=temp[['POZYCJA',2023,2024,2025,2026]].copy()
tabela=tabela.style.format({2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'BP_1.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2027,2028,2029,2030]].copy()
tabela=tabela.style.format({2027:'{0:,.2f}',2028:'{0:,.2f}',2029:'{0:,.2f}',2030:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'BP_2.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2031,2032,2033,2034]].copy()
tabela=tabela.style.format({2031:'{0:,.2f}',2032:'{0:,.2f}',2033:'{0:,.2f}',2034:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'BP_3.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

In [270]:
# Cash Flow Inwestycyjny- Tabela

t_cfi=t_model[['R','A. WPŁYWY PIENIĘŻNE OGÓŁEM', '1. Kapitał zakładowy',
       '2. Kredyty bankowe', '3. Leasing finansowy', '4. Inne',
       '5. Przychody ze sprzedaży', 'B. WYDATKI PIENIĘŻNE OGÓŁEM',
       '1. Nakłady inwestycyjne', '2. Koszty operacyjne (bez amortyzacji)',
       '3. Pozostałe koszty operacyjne', '4. Spłaty kredytów i leasingów',
       '5. Odsetki', '6. Podatek dochodowy', 'C. ZAPOTRZEBOWANIE NA KON',
       'D. ZMIANA ŚRODKÓW PIENIĘŻNYCH']].copy()

t_cfi['A. WPŁYWY PIENIĘŻNE OGÓŁEM']=t_cfi['A. WPŁYWY PIENIĘŻNE OGÓŁEM']-t_cfi['1. Kapitał zakładowy']-t_cfi['2. Kredyty bankowe']-t_cfi['3. Leasing finansowy']-t_cfi['4. Inne']
t_cfi['B. WYDATKI PIENIĘŻNE OGÓŁEM']=t_cfi['B. WYDATKI PIENIĘŻNE OGÓŁEM']-t_cfi['5. Odsetki']-t_cfi['4. Spłaty kredytów i leasingów']
t_cfi['D. ZMIANA ŚRODKÓW PIENIĘŻNYCH']=t_cfi['D. ZMIANA ŚRODKÓW PIENIĘŻNYCH']-t_cfi['1. Kapitał zakładowy']-t_cfi['2. Kredyty bankowe']-t_cfi['3. Leasing finansowy']-t_cfi['4. Inne']-t_cfi['5. Odsetki']-t_cfi['4. Spłaty kredytów i leasingów']
t_cfi['1. Kapitał zakładowy']=0
t_cfi['2. Kredyty bankowe']=0
t_cfi['3. Leasing finansowy']=0
t_cfi['4. Inne']=0
t_cfi['5. Odsetki']=0
t_cfi['4. Spłaty kredytów i leasingów']=0

t_cfi['E. DCF (Discounted Cash Flow)']=t_cfi['D. ZMIANA ŚRODKÓW PIENIĘŻNYCH']/(1+DISCOUNT_RATE/1200)**t_cfi.index

t_fin=t_cfi['D. ZMIANA ŚRODKÓW PIENIĘŻNYCH']

t_cfi=t_cfi.groupby(['R'],as_index=False)[['A. WPŁYWY PIENIĘŻNE OGÓŁEM', '1. Kapitał zakładowy',
       '2. Kredyty bankowe', '3. Leasing finansowy', '4. Inne',
       '5. Przychody ze sprzedaży', 'B. WYDATKI PIENIĘŻNE OGÓŁEM',
       '1. Nakłady inwestycyjne', '2. Koszty operacyjne (bez amortyzacji)',
       '3. Pozostałe koszty operacyjne', '4. Spłaty kredytów i leasingów',
       '5. Odsetki', '6. Podatek dochodowy', 'C. ZAPOTRZEBOWANIE NA KON',
       'D. ZMIANA ŚRODKÓW PIENIĘŻNYCH','E. DCF (Discounted Cash Flow)']].sum()

t_cfi['F. NPV (SKUMULOWANE DCF)']=t_cfi['E. DCF (Discounted Cash Flow)'].cumsum()

t_cfi.rename(columns={'R':'POZYCJA'},inplace=True)
t_cfi=t_cfi.set_index('POZYCJA')

t_cfi=t_cfi/1000
t_cfi=t_cfi.T
t_cfi=t_cfi.reset_index()
t_cfi=pd.DataFrame(t_cfi.values.tolist(),columns=['POZYCJA',2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034])
t_cfi

,POZYCJA,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034
0,A. WPŁYWY PIENIĘŻNE OGÓŁEM,"6,540.05","6,501.97","6,712.13","6,649.94","6,726.16","6,649.79","6,592.18","6,809.96","6,755.56","6,839.71","7,062.44","7,013.09"
1,1. Kapitał zakładowy,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2. Kredyty bankowe,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,3. Leasing finansowy,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,4. Inne,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,5. Przychody ze sprzedaży,"6,540.05","6,501.97","6,712.13","6,649.94","6,726.16","6,649.79","6,592.18","6,809.96","6,755.56","6,839.71","7,062.44","7,013.09"
6,B. WYDATKI PIENIĘŻNE OGÓŁEM,"-13,346.18","-5,112.96","-5,331.42","-5,172.86","-5,203.71","-5,423.98","-5,267.27","-5,488.81","-5,333.40","-5,367.46","-5,591.00","-5,437.64"
7,1. Nakłady inwestycyjne,"-8,200.53",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,2. Koszty operacyjne (bez amortyzacji),"-5,145.65","-5,112.96","-5,331.42","-5,172.86","-5,203.71","-5,423.98","-5,267.27","-5,488.81","-5,333.40","-5,367.46","-5,591.00","-5,437.64"
9,3. Pozostałe koszty operacyjne,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [271]:
# Tabela do raportu - wartości
temp=t_cfi

tabela=temp[['POZYCJA',2023,2024,2025,2026]].copy()
tabela=tabela.style.format({2023:'{0:,.2f}',2024:'{0:,.2f}',2025:'{0:,.2f}',2026:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'CFI_1.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2027,2028,2029,2030]].copy()
tabela=tabela.style.format({2027:'{0:,.2f}',2028:'{0:,.2f}',2029:'{0:,.2f}',2030:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'CFI_2.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

tabela=temp[['POZYCJA',2031,2032,2033,2034]].copy()
tabela=tabela.style.format({2031:'{0:,.2f}',2032:'{0:,.2f}',2033:'{0:,.2f}',2034:'{0:,.2f}'}).hide(axis='index')
tabela.to_latex(ASSETS / 'CFI_3.tex',hrules=True,column_format='L{7cm}R{1.5cm}R{1.5cm}R{1.5cm}R{1.5cm}')

In [272]:
Wyniki=pd.DataFrame([['IRR',npf.irr(t_fin)*100*12],['NPV',npf.npv(0.09/12,t_fin)]],columns=['Miara','Wynik'])
Wyniki

,Miara,Wynik
0,IRR,14.23
1,NPV,"2,238,258.65"
